In [2]:
import cv2
import numpy as np
import keyboard
from collections import deque

class DrawObject():
    points = deque(maxlen = 512)
    def __init__(self,tool,color,size):
        self.tool = tool
        self.color = color
        self.size = size
        self.points = deque(maxlen = 512)
    
    def getPoints(self):
        return self.points
    
    def getTool(self):
        return self.tool
    
    def addPoint(self,pnt):
        self.points.appendleft(pnt)
        
    def draw(self,camera,canvas):
        if self.tool == 0:
            for j in range(1,len(self.points)):
                if self.points[j-1] is not None and self.points[j] is not None:
                    cv2.line(camera, self.points[j-1], self.points[j], self.color, self.size)
                    cv2.line(canvas, self.points[j-1], self.points[j], self.color, self.size)
            

        
def setValues(x):
    print("")

oldX = -1
oldY = -1
newX = -1
newY = -1    
paints = [DrawObject(0,(0,0,0),2)]
paintIndex = 0
    
kernel = np.ones((5, 5), np.uint8)

cv2.namedWindow("Color Detection")
cv2.createTrackbar("UpperHue", "Color Detection",
                   90, 180, setValues)
cv2.createTrackbar("UpperSat", "Color Detection",
                   255, 255, setValues)
cv2.createTrackbar("UpperVal", "Color Detection",
                   255, 255, setValues)
cv2.createTrackbar("LowerHue", "Color Detection",
                   60, 180, setValues)
cv2.createTrackbar("LowerSat", "Color Detection",
                   72, 255, setValues)
cv2.createTrackbar("LowerVal", "Color Detection",
                   49, 255, setValues)


# Create videocapture object
cap = cv2.VideoCapture(0)

size = cap.read()[1].shape

paint = np.zeros(size) + 255

cv2.createTrackbar("UpperRad", "Color Detection",
                   75, min([size[0],size[1]]), setValues)
cv2.createTrackbar("LowerRad", "Color Detection",
                   25, min([size[0],size[1]]), setValues)

cv2.resizeWindow("Color Detection",400,200)

while True:
    if keyboard.is_pressed("c"):
        paints = [DrawObject(0,(0,0,0),2)]
        paintIndex = 0
    
    # Read each frame from webcam
    success, cam = cap.read()
    cam = cv2.flip(cam, 1)
    hsv = cv2.cvtColor(cam, cv2.COLOR_BGR2HSV)
    
    upHue = cv2.getTrackbarPos("UpperHue","Color Detection")
    upSat = cv2.getTrackbarPos("UpperSat","Color Detection")
    upVal = cv2.getTrackbarPos("UpperVal","Color Detection")
    lowHue = cv2.getTrackbarPos("LowerHue","Color Detection")
    lowSat = cv2.getTrackbarPos("LowerSat","Color Detection")
    lowVal = cv2.getTrackbarPos("LowerVal","Color Detection")
    
    upperRad = cv2.getTrackbarPos("UpperRad", "Color Detection")
    lowerRad = cv2.getTrackbarPos("LowerRad", "Color Detection")
    
    upHSV = np.array([upHue,upSat,upVal])
    lowHSV = np.array([lowHue,lowSat,lowVal])
    
    mask = cv2.inRange(hsv, lowHSV, upHSV)
    mask = cv2.erode(mask, kernel, iterations = 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations = 1)
    
    cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    center = None

    
    if(len(cnts) > 0):
        
        # Getting the center point
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        
        cv2.circle(cam, (int(x), int(y)), int(radius), (255, 255, 0) if (radius < upperRad and radius > lowerRad) else (0, 0, 255), 2)
    
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

        # Incrementing points
        if(radius < upperRad and radius > lowerRad):
            paints[paintIndex].addPoint(center)
        elif len(paints[paintIndex].getPoints()) > 0:
            paints.append(DrawObject(0,(0,0,0) if paintIndex %2 == 0 else (0,255,255) ,2))
            paintIndex += 1
    elif len(paints[paintIndex].getPoints()) > 0:
        paints.append(DrawObject(0,(0,0,0),2))
        paintIndex += 1
    
    # Drawing line
    for i in paints:
        i.draw(cam,paint)
    
    cv2.imshow("Camera", cam)
    cv2.imshow("Paint", paint)
    
    # Open the OpenCV window until 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()


cv2.destroyAllWindows()